# Download COVID-19 related data from https://ourworldindata.org for visualization.

## 1. Initialization

### 1.1 Define useful paths in the project.

In [1]:
import os
proj_dir = os.getcwd()
data_dir = os.path.join(proj_dir, 'data')
jhu_dir = os.path.join(data_dir, 'jhu')
data_output_dir = os.path.join(data_dir, 'output')

### 1.2 Import libraries

In [2]:
import pandas as pd
from datetime import datetime

### 1.3 Download the datasets

In [3]:
def download_csv():
    version = datetime.now().strftime('%Y%m%d-%H%M%S')
    dest = os.path.join(jhu_dir, version)
    if os.path.exists(dest):
        return
    else:
        os.mkdir(os.path.join(jhu_dir, version))
        url_confirmed = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
        url_confirmed_us = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
        url_deaths = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
        url_deaths_us = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
        url_recovered = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
        url_iso_lookup = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv'
        
        files = {
            'confirmed': url_confirmed,
            'confirmed_us': url_confirmed_us,
            'deaths': url_deaths, 
            'deaths_us': url_deaths_us,
            'recovered': url_recovered,
            'iso_lookup': url_iso_lookup
        }
        
        for k in files:
            data = pd.read_csv(files[k], index_col=False)
            data.to_csv(os.path.join(dest, k+'.csv'), index=False)

In [4]:
download_csv()
latest_folder = [d for d in sorted(os.listdir(jhu_dir)) if os.path.isdir(os.path.join(jhu_dir, d))][-1]
latest_folder

'20220328-115912'

## 2.   Process Data

### 2.1 Load datasets to pandas DataFrames

In [5]:
df_cases_global_untouched = pd.read_csv(os.path.join(jhu_dir, latest_folder, 'confirmed.csv'))
df_iso_cnty_untouched = pd.read_csv(os.path.join(jhu_dir, latest_folder, 'iso_lookup.csv'))

### 2.2 Transform data

#### 2.2.1  Countries Infomation

1. Copy to a DataFrame named ```df_iso_cnty``` from the untouched copy of the data.
2. Examine the structure.
    -  ```Combined_Key``` combines ```Province_State``` and ```Country_Region``` if available.

In [6]:
df_iso_cnty = df_iso_cnty_untouched.copy()
df_iso_cnty.head(20)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
0,4,AF,AFG,4.0,NaN,NaN,NaN,Afghanistan,33.939110,67.709953,Afghanistan,38928341.0
1,8,AL,ALB,8.0,NaN,NaN,NaN,Albania,41.153300,20.168300,Albania,2877800.0
2,10,AQ,ATA,10.0,NaN,NaN,NaN,Antarctica,-71.949900,23.347000,Antarctica,NaN
3,12,DZ,DZA,12.0,NaN,NaN,NaN,Algeria,28.033900,1.659600,Algeria,43851043.0
4,20,AD,AND,20.0,NaN,NaN,NaN,Andorra,42.506300,1.521800,Andorra,77265.0
5,24,AO,AGO,24.0,NaN,NaN,NaN,Angola,-11.202700,17.873900,Angola,32866268.0
6,28,AG,ATG,28.0,NaN,NaN,NaN,Antigua and Barbuda,17.060800,-61.796400,Antigua and Barbuda,97928.0
7,32,AR,ARG,32.0,NaN,NaN,NaN,Argentina,-38.416100,-63.616700,Argentina,45195777.0
8,51,AM,ARM,51.0,NaN,NaN,NaN,Armenia,40.069100,45.038200,Armenia,2963234.0
9,40,AT,AUT,40.0,NaN,NaN,NaN,Austria,47.516200,14.550100,Austria,9006400.0


#### 2.2.2  COVID-19 Cases Data

1. Copy to a DataFrame named ```df_cases_global``` from the untouched copy of the data.

In [7]:
df_cases_global = df_cases_global_untouched.copy()
df_cases_global

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/18/22,3/19/22,3/20/22,3/21/22,3/22/22,3/23/22,3/24/22,3/25/22,3/26/22,3/27/22
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,176983,177039,177093,177191,177255,177321,177321,177321,177321,177520
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,272961,273040,273088,273088,273146,273164,273257,273318,273387,273432
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,265524,265539,265550,265562,265573,265585,265599,265612,265621,265629
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,39234,39234,39234,39234,39234,39713,39713,39713,39713,39713
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,99003,99003,99003,99010,99058,99058,99081,99102,99106,99115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,654380,654380,654380,654954,654954,654954,655468,655468,655468,655468
280,NaN,Winter Olympics 2022,39.904200,116.407400,0,0,0,0,0,0,...,535,535,535,535,535,535,535,535,535,535
281,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,11797,11799,11801,11801,11802,11802,11803,11803,11803,11803
282,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,315623,315623,315623,315892,316088,316190,316312,316422,316501,316550


2. Drop the unnecessary columns (i.e. ```Lat``` and ```Long```) which are included in Countries Information.

In [8]:
df_cases_global = df_cases_global.drop(columns=['Lat', 'Long'])
df_cases_global

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,3/18/22,3/19/22,3/20/22,3/21/22,3/22/22,3/23/22,3/24/22,3/25/22,3/26/22,3/27/22
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,176983,177039,177093,177191,177255,177321,177321,177321,177321,177520
1,NaN,Albania,0,0,0,0,0,0,0,0,...,272961,273040,273088,273088,273146,273164,273257,273318,273387,273432
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,265524,265539,265550,265562,265573,265585,265599,265612,265621,265629
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,39234,39234,39234,39234,39234,39713,39713,39713,39713,39713
4,NaN,Angola,0,0,0,0,0,0,0,0,...,99003,99003,99003,99010,99058,99058,99081,99102,99106,99115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,NaN,West Bank and Gaza,0,0,0,0,0,0,0,0,...,654380,654380,654380,654954,654954,654954,655468,655468,655468,655468
280,NaN,Winter Olympics 2022,0,0,0,0,0,0,0,0,...,535,535,535,535,535,535,535,535,535,535
281,NaN,Yemen,0,0,0,0,0,0,0,0,...,11797,11799,11801,11801,11802,11802,11803,11803,11803,11803
282,NaN,Zambia,0,0,0,0,0,0,0,0,...,315623,315623,315623,315892,316088,316190,316312,316422,316501,316550


3. Create a column ```key``` as the key to match with ```df_cnty_info```.

In [9]:
df_cases_global['key'] = df_cases_global.apply(lambda x: x['Country/Region'] if pd.isna(x['Province/State']) else x['Province/State'] + ', ' + x['Country/Region'], axis=1)
df_cases_global

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,3/19/22,3/20/22,3/21/22,3/22/22,3/23/22,3/24/22,3/25/22,3/26/22,3/27/22,key
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,177039,177093,177191,177255,177321,177321,177321,177321,177520,Afghanistan
1,NaN,Albania,0,0,0,0,0,0,0,0,...,273040,273088,273088,273146,273164,273257,273318,273387,273432,Albania
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,265539,265550,265562,265573,265585,265599,265612,265621,265629,Algeria
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,39234,39234,39234,39234,39713,39713,39713,39713,39713,Andorra
4,NaN,Angola,0,0,0,0,0,0,0,0,...,99003,99003,99010,99058,99058,99081,99102,99106,99115,Angola
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,NaN,West Bank and Gaza,0,0,0,0,0,0,0,0,...,654380,654380,654954,654954,654954,655468,655468,655468,655468,West Bank and Gaza
280,NaN,Winter Olympics 2022,0,0,0,0,0,0,0,0,...,535,535,535,535,535,535,535,535,535,Winter Olympics 2022
281,NaN,Yemen,0,0,0,0,0,0,0,0,...,11799,11801,11801,11802,11802,11803,11803,11803,11803,Yemen
282,NaN,Zambia,0,0,0,0,0,0,0,0,...,315623,315623,315892,316088,316190,316312,316422,316501,316550,Zambia


4. Merge the cases data ```df_cases_global``` with countries information ```df_iso_cnty``` to include the 3-letter country code ```iso3``` in the data.

In [10]:
df_cases_global = pd.merge(df_cases_global, df_iso_cnty[['iso3', 'Combined_Key']], how='left', left_on='key', right_on='Combined_Key')
df_cases_global

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,3/21/22,3/22/22,3/23/22,3/24/22,3/25/22,3/26/22,3/27/22,key,iso3,Combined_Key
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,177191,177255,177321,177321,177321,177321,177520,Afghanistan,AFG,Afghanistan
1,NaN,Albania,0,0,0,0,0,0,0,0,...,273088,273146,273164,273257,273318,273387,273432,Albania,ALB,Albania
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,265562,265573,265585,265599,265612,265621,265629,Algeria,DZA,Algeria
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,39234,39234,39713,39713,39713,39713,39713,Andorra,AND,Andorra
4,NaN,Angola,0,0,0,0,0,0,0,0,...,99010,99058,99058,99081,99102,99106,99115,Angola,AGO,Angola
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,NaN,West Bank and Gaza,0,0,0,0,0,0,0,0,...,654954,654954,654954,655468,655468,655468,655468,West Bank and Gaza,PSE,West Bank and Gaza
280,NaN,Winter Olympics 2022,0,0,0,0,0,0,0,0,...,535,535,535,535,535,535,535,Winter Olympics 2022,NaN,Winter Olympics 2022
281,NaN,Yemen,0,0,0,0,0,0,0,0,...,11801,11802,11802,11803,11803,11803,11803,Yemen,YEM,Yemen
282,NaN,Zambia,0,0,0,0,0,0,0,0,...,315892,316088,316190,316312,316422,316501,316550,Zambia,ZMB,Zambia


5. Examine any ```NaN``` value for ```iso3```.

In [11]:
df_cases_global[df_cases_global['iso3'].isna()]

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,3/21/22,3/22/22,3/23/22,3/24/22,3/25/22,3/26/22,3/27/22,key,iso3,Combined_Key
47,Northwest Territories,Canada,0,0,0,0,0,0,0,0,...,9942,10005,10106,10235,10333,10333,10333,"Northwest Territories, Canada",NaN,NaN
53,Repatriated Travellers,Canada,0,0,0,0,0,0,0,0,...,13,13,13,13,13,13,13,"Repatriated Travellers, Canada",NaN,NaN
106,NaN,Diamond Princess,0,0,0,0,0,0,0,0,...,712,712,712,712,712,712,712,Diamond Princess,NaN,Diamond Princess
174,NaN,MS Zaandam,0,0,0,0,0,0,0,0,...,9,9,9,9,9,9,9,MS Zaandam,NaN,MS Zaandam
241,NaN,Summer Olympics 2020,0,0,0,0,0,0,0,0,...,865,865,865,865,865,865,865,Summer Olympics 2020,NaN,Summer Olympics 2020
280,NaN,Winter Olympics 2022,0,0,0,0,0,0,0,0,...,535,535,535,535,535,535,535,Winter Olympics 2022,NaN,Winter Olympics 2022


6. Apply a fix for the two rows (i.e. #47 and #53) which the ```iso3``` should be ```CAN``` (i.e. Canada).

In [12]:
cond_CAN = (df_cases_global['iso3'].isna()) & (df_cases_global['Country/Region']=='Canada')
df_cases_global.loc[cond_CAN, 'iso3'] = 'CAN'

7. Examine the data for any more ```NaN``` for ```iso3```.

In [13]:
df_cases_global[df_cases_global['iso3'].isna()]

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,3/21/22,3/22/22,3/23/22,3/24/22,3/25/22,3/26/22,3/27/22,key,iso3,Combined_Key
106,NaN,Diamond Princess,0,0,0,0,0,0,0,0,...,712,712,712,712,712,712,712,Diamond Princess,NaN,Diamond Princess
174,NaN,MS Zaandam,0,0,0,0,0,0,0,0,...,9,9,9,9,9,9,9,MS Zaandam,NaN,MS Zaandam
241,NaN,Summer Olympics 2020,0,0,0,0,0,0,0,0,...,865,865,865,865,865,865,865,Summer Olympics 2020,NaN,Summer Olympics 2020
280,NaN,Winter Olympics 2022,0,0,0,0,0,0,0,0,...,535,535,535,535,535,535,535,Winter Olympics 2022,NaN,Winter Olympics 2022


8. Remove any rows which is not a country or region with valid ```iso3```.

In [14]:
df_cases_global = df_cases_global.dropna(subset='iso3')
df_cases_global

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,3/21/22,3/22/22,3/23/22,3/24/22,3/25/22,3/26/22,3/27/22,key,iso3,Combined_Key
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,177191,177255,177321,177321,177321,177321,177520,Afghanistan,AFG,Afghanistan
1,NaN,Albania,0,0,0,0,0,0,0,0,...,273088,273146,273164,273257,273318,273387,273432,Albania,ALB,Albania
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,265562,265573,265585,265599,265612,265621,265629,Algeria,DZA,Algeria
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,39234,39234,39713,39713,39713,39713,39713,Andorra,AND,Andorra
4,NaN,Angola,0,0,0,0,0,0,0,0,...,99010,99058,99058,99081,99102,99106,99115,Angola,AGO,Angola
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,NaN,Vietnam,0,2,2,2,2,2,2,2,...,8089761,8338914,8479751,8599751,8761252,8919557,9011473,Vietnam,VNM,Vietnam
279,NaN,West Bank and Gaza,0,0,0,0,0,0,0,0,...,654954,654954,654954,655468,655468,655468,655468,West Bank and Gaza,PSE,West Bank and Gaza
281,NaN,Yemen,0,0,0,0,0,0,0,0,...,11801,11802,11802,11803,11803,11803,11803,Yemen,YEM,Yemen
282,NaN,Zambia,0,0,0,0,0,0,0,0,...,315892,316088,316190,316312,316422,316501,316550,Zambia,ZMB,Zambia


9. Some of the countries have a breakdown of the number of cases in different provinces or states. Sum up the number of cases by country (i.e. the same ```iso3```).

In [15]:
df_cases_cnty = df_cases_global.groupby('iso3', as_index=False).sum()
df_cases_cnty

,iso3,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,3/18/22,3/19/22,3/20/22,3/21/22,3/22/22,3/23/22,3/24/22,3/25/22,3/26/22,3/27/22
0,ABW,0,0,0,0,0,0,0,0,0,...,33843,33843,33843,33843,33843,33843,33843,33929,33929,33929
1,AFG,0,0,0,0,0,0,0,0,0,...,176983,177039,177093,177191,177255,177321,177321,177321,177321,177520
2,AGO,0,0,0,0,0,0,0,0,0,...,99003,99003,99003,99010,99058,99058,99081,99102,99106,99115
3,AIA,0,0,0,0,0,0,0,0,0,...,2595,2595,2595,2595,2633,2633,2633,2633,2633,2633
4,ALB,0,0,0,0,0,0,0,0,0,...,272961,273040,273088,273088,273146,273164,273257,273318,273387,273432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,XKS,0,0,0,0,0,0,0,0,0,...,227057,227093,227124,227150,227191,227231,227265,227296,227324,227361
222,YEM,0,0,0,0,0,0,0,0,0,...,11797,11799,11801,11801,11802,11802,11803,11803,11803,11803
223,ZAF,0,0,0,0,0,0,0,0,0,...,3700484,3703329,3704218,3704784,3705696,3705696,3709209,3710766,3712263,3713252
224,ZMB,0,0,0,0,0,0,0,0,0,...,315623,315623,315623,315892,316088,316190,316312,316422,316501,316550


#### 2.2.3  Countries Infomation (continued)

3. Examine the structure of the data.

In [16]:
# df_iso_cnty = df_iso_cnty_untouched.copy()
df_iso_cnty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4219 entries, 0 to 4218
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   UID             4219 non-null   int64  
 1   iso2            4214 non-null   object 
 2   iso3            4215 non-null   object 
 3   code3           4215 non-null   float64
 4   FIPS            3384 non-null   float64
 5   Admin2          3343 non-null   object 
 6   Province_State  4020 non-null   object 
 7   Country_Region  4219 non-null   object 
 8   Lat             4074 non-null   float64
 9   Long_           4074 non-null   float64
 10  Combined_Key    4219 non-null   object 
 11  Population      4071 non-null   float64
dtypes: float64(5), int64(1), object(6)
memory usage: 395.7+ KB


4. Remove any rows without valid ```iso3``` (i.e. ```NaN```).
5. Sort the DataFrame ```df_iso_cnty``` so that the row describing a country comes first if ```Province_State``` data is available.
    - Example:
        |...    |iso3   |...    |Province_State |Country_Region |...    |Combined_Key   |...    |
        |:------|:------|:------|:--------------|:--------------|:------|:--------------|:------|
        |...    |...    |...    |...            |...            |...    |...            |...    |
        |...    |CAN    |...    |NaN            |Canada         |...    |Canada         |...    |
        |...    |CAN    |...    |Ontario        |Canada         |...    |Ontario, Canada|...    |
        |...    |...    |...    |...            |...            |...    |...            |...    |
6.  Remove any rows with the same ```iso3``` and keep the first row.

In [17]:
df_iso_cnty = df_iso_cnty.dropna(subset='iso3')
df_iso_cnty = df_iso_cnty.sort_values(by=['Country_Region', 'iso3', 'Province_State'], na_position='first')
df_iso_cnty = df_iso_cnty.drop_duplicates('iso3', keep='first')
df_iso_cnty

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
0,4,AF,AFG,4.0,NaN,NaN,NaN,Afghanistan,33.939110,67.709953,Afghanistan,38928341.0
1,8,AL,ALB,8.0,NaN,NaN,NaN,Albania,41.153300,20.168300,Albania,2877800.0
3,12,DZ,DZA,12.0,NaN,NaN,NaN,Algeria,28.033900,1.659600,Algeria,43851043.0
4,20,AD,AND,20.0,NaN,NaN,NaN,Andorra,42.506300,1.521800,Andorra,77265.0
5,24,AO,AGO,24.0,NaN,NaN,NaN,Angola,-11.202700,17.873900,Angola,32866268.0
...,...,...,...,...,...,...,...,...,...,...,...,...
747,275,PS,PSE,275.0,NaN,NaN,NaN,West Bank and Gaza,31.952200,35.233200,West Bank and Gaza,5101416.0
748,732,EH,ESH,732.0,NaN,NaN,NaN,Western Sahara,24.215500,-12.885800,Western Sahara,597330.0
749,887,YE,YEM,887.0,NaN,NaN,NaN,Yemen,15.552727,48.516388,Yemen,29825968.0
750,894,ZM,ZMB,894.0,NaN,NaN,NaN,Zambia,-13.133897,27.849332,Zambia,18383956.0


7. Drop unused columns and rename the remaining columns.

In [18]:
df_iso_cnty = df_iso_cnty[['iso3', 'Combined_Key', 'Lat', 'Long_', 'Population']]
df_iso_cnty.columns = ['iso3', 'Country/Region', 'Latitude', 'Longitude', 'Population']
df_iso_cnty

,iso3,Country/Region,Latitude,Longitude,Population
0,AFG,Afghanistan,33.939110,67.709953,38928341.0
1,ALB,Albania,41.153300,20.168300,2877800.0
3,DZA,Algeria,28.033900,1.659600,43851043.0
4,AND,Andorra,42.506300,1.521800,77265.0
5,AGO,Angola,-11.202700,17.873900,32866268.0
...,...,...,...,...,...
747,PSE,West Bank and Gaza,31.952200,35.233200,5101416.0
748,ESH,Western Sahara,24.215500,-12.885800,597330.0
749,YEM,Yemen,15.552727,48.516388,29825968.0
750,ZMB,Zambia,-13.133897,27.849332,18383956.0


### 2.3 Merge COVID-19 Cases Data with Countries Information
1. Merge the two DataFrames on the common key ```iso3```.
2. Set ```iso3``` as the index.

In [19]:
df_full_data = pd.merge(df_iso_cnty, df_cases_cnty, how='right', on='iso3')
df_full_data

,iso3,Country/Region,Latitude,Longitude,Population,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,3/18/22,3/19/22,3/20/22,3/21/22,3/22/22,3/23/22,3/24/22,3/25/22,3/26/22,3/27/22
0,ABW,"Aruba, Netherlands",12.521100,-69.968300,106766.0,0,0,0,0,0,...,33843,33843,33843,33843,33843,33843,33843,33929,33929,33929
1,AFG,Afghanistan,33.939110,67.709953,38928341.0,0,0,0,0,0,...,176983,177039,177093,177191,177255,177321,177321,177321,177321,177520
2,AGO,Angola,-11.202700,17.873900,32866268.0,0,0,0,0,0,...,99003,99003,99003,99010,99058,99058,99081,99102,99106,99115
3,AIA,"Anguilla, United Kingdom",18.220600,-63.068600,15002.0,0,0,0,0,0,...,2595,2595,2595,2595,2633,2633,2633,2633,2633,2633
4,ALB,Albania,41.153300,20.168300,2877800.0,0,0,0,0,0,...,272961,273040,273088,273088,273146,273164,273257,273318,273387,273432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,XKS,Kosovo,42.602636,20.902977,1810366.0,0,0,0,0,0,...,227057,227093,227124,227150,227191,227231,227265,227296,227324,227361
222,YEM,Yemen,15.552727,48.516388,29825968.0,0,0,0,0,0,...,11797,11799,11801,11801,11802,11802,11803,11803,11803,11803
223,ZAF,South Africa,-30.559500,22.937500,59308690.0,0,0,0,0,0,...,3700484,3703329,3704218,3704784,3705696,3705696,3709209,3710766,3712263,3713252
224,ZMB,Zambia,-13.133897,27.849332,18383956.0,0,0,0,0,0,...,315623,315623,315623,315892,316088,316190,316312,316422,316501,316550


In [20]:
df_full_data = df_full_data.set_index('iso3')
df_full_data

,Country/Region,Latitude,Longitude,Population,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/18/22,3/19/22,3/20/22,3/21/22,3/22/22,3/23/22,3/24/22,3/25/22,3/26/22,3/27/22
iso3,,,,,,,,,,,,,,,,,,,,,
ABW,"Aruba, Netherlands",12.521100,-69.968300,106766.0,0,0,0,0,0,0,...,33843,33843,33843,33843,33843,33843,33843,33929,33929,33929
AFG,Afghanistan,33.939110,67.709953,38928341.0,0,0,0,0,0,0,...,176983,177039,177093,177191,177255,177321,177321,177321,177321,177520
AGO,Angola,-11.202700,17.873900,32866268.0,0,0,0,0,0,0,...,99003,99003,99003,99010,99058,99058,99081,99102,99106,99115
AIA,"Anguilla, United Kingdom",18.220600,-63.068600,15002.0,0,0,0,0,0,0,...,2595,2595,2595,2595,2633,2633,2633,2633,2633,2633
ALB,Albania,41.153300,20.168300,2877800.0,0,0,0,0,0,0,...,272961,273040,273088,273088,273146,273164,273257,273318,273387,273432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XKS,Kosovo,42.602636,20.902977,1810366.0,0,0,0,0,0,0,...,227057,227093,227124,227150,227191,227231,227265,227296,227324,227361
YEM,Yemen,15.552727,48.516388,29825968.0,0,0,0,0,0,0,...,11797,11799,11801,11801,11802,11802,11803,11803,11803,11803
ZAF,South Africa,-30.559500,22.937500,59308690.0,0,0,0,0,0,0,...,3700484,3703329,3704218,3704784,3705696,3705696,3709209,3710766,3712263,3713252


## 3.   Export the transformed data

In [21]:
df_full_data.to_csv(os.path.join(data_output_dir, 'cnty_total_confirmed_cases_{}.csv'.format(latest_folder)))